## This file is created to plot the 3D figures of the article "GNN graph structures in network anomaly detection" (NOMS 2025)

In [1]:
#imports
import os
import random
import numpy as np
import pandas as pd
from timeit import default_timer as timer
import time
import re
import Tools_first_step as tfs

import matplotlib.pyplot as plt
from matplotlib import cm 
from matplotlib.ticker import LinearLocator
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

pd.options.mode.chained_assignment = None

In [2]:
#loading data set
df=pd.read_csv('/home/student/Documents/Data/cisco/preprocessed_cisco_train.csv')
#pd.read_csv('/home/student/Documents/Data/Training_Set_Day_1/dataset_end_total_preprocessed.csv') #no more delimiter to change after this preprocessing
columns=df.columns
dataset="cisco"

In [ ]:
def rmv_Unnamed(columns):

    str_remove_go="Unnamed"+".*"
    regex_str=re.compile(str_remove_go)
    not_good_col=list(filter(regex_str.match, columns))
    col_keeper=[]
    for i in columns:
        if i not in not_good_col:
            col_keeper.append(i)
        
    new_list=list(col_keeper)

    return new_list

col_to_keep=rmv_Unnamed(df.columns)

df=df[col_to_keep]
print(df.shape)

In [7]:
name_columns = df.columns
size_mode=25

In [8]:
from gensim.models import Word2Vec

def dimentionality():
    dim = []
    x = []
    

    for i in range(3, size_mode+4):
        model = Word2Vec(sentences = tfs.split_all(name_columns,dt=dataset), vector_size= i, window=5, min_count=1, workers=4, epochs=50)
        #vector_size = 25, min_count=1, workers = 4)
        dim.append(model)
        x.append(model.vector_size)
    for i in range(len(dim)):
        model = dim[i]
        vectors = model.wv
        dim.append(vectors)

    return dim, x 
        
                   

In [ ]:
my_dimention= dimentionality()
print(my_dimention)

In [10]:

#first
def threashold():
    y = threashold = []
    initial = 0.1
    y.append(initial)
    for _ in range(size_mode):
        initial += 0.035
        y.append(round(initial,3))

    return y

def threashold_v2():
    y = threashold = []
    initial = 0.7
    y.append(initial)
    for _ in range(size_mode):
        initial += 0.012
        y.append(round(initial,3))

    return y

In [ ]:
threashold_result = threashold_v2()
print(threashold_result)

In [12]:
cosine_similarities = tfs.score_similarity_current(name_columns, my_dimention[0][0], dt=dataset)

In [15]:
corr_mat=df.corr()

In [16]:
dico_correlation=tfs.change_into_dicto(corr_mat)

In [17]:
My_dictionary=tfs.build_score_lambda(dico_correlation, cosine_similarities, 0.5) #no plot in >3D warning
My_dictionary_25=tfs.build_score_lambda(dico_correlation, cosine_similarities, 0.5)
my_edges_test= tfs.edges_dico(My_dictionary, threashold)
my_edges_test_25=tfs.edges_dico(My_dictionary_25, threashold)
#useless1,useless2,useless2, edges_complet=tfs.get_edges_v3(My_dictionary,threashold,vectores_places,"server")
#useless4,useless5,useless6, edges_complet_25=tfs.get_edges_v3(My_dictionary_25,threashold,vectores_places,"server") 

In [18]:
def total_edges():
    edges = []
    
    first_list= my_dimention[0]
    first_list=first_list[:len(first_list)//2] # remove half of the values at the end of the list that are not word2vec models
    count_my_dim=2
    for model in first_list:
        count_my_dim+=1
        a = tfs.score_similarity_current(name_columns , model, dt="cisco")
        #d_new=tfs.build_score_lambda(dico_correlation, a[0],0.5)
        for i in threashold_result:
            
            print(f'threashold = {i} -- model dimension={count_my_dim}')
            total_number = tfs.get_edges_v2(a[0], i, a[1])
            edges.append(total_number[4])
            

    return edges
        
          

In [ ]:
total_edges = total_edges()
print(total_edges)

In [ ]:
print(total_edges)

In [21]:
def average_edges():
    edges = []
    for i in range(len(total_edges)):
        edges.append(tfs.average_number_of_edges(df.shape[1], total_edges[i]))
    return edges

In [ ]:
ave_edges = average_edges()
print(ave_edges)

In [ ]:
#this is the plot
#threashold_result2= threashold_result[::-1]
#print(threashold_result)
#ave_edges_log=np.log(ave_edges)

print(len(threashold_result))
print(len(ave_edges))
print(len(my_dimention[1]))
y = np.array(my_dimention[1])
x = np.array(threashold_result)#[::-1])
z = np.array(ave_edges)

X, Y = np.meshgrid(x,y)
#Z = np.tile(z, (len(y), 1))
Z = np.reshape(z, (len(x), len(y)))
print(Z[0])
print(Z[-3])
fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')

surf = ax.plot_surface(X, Y, Z, cmap = 'copper')
#ax.view_init(azim=45)
ax.set_xlabel('Threashold')
ax.set_ylabel('dimentionality')
ax.set_zlabel('Average edges')

ax.set_ylim(28,3)
#ax.set_xlim(28,3)
fig.colorbar(surf)
plt.show()

